### Using Fractions

In [2]:
import os, sys, copy

import numpy as np
import pandas as pd

from fractions import Fraction
from tabulate import tabulate

from helper import visualize_tabulation, all_slack_starting

In [11]:
arr1 = np.array([1, 2])
arr2 = np.array([2, 3])

frac_arr1 = np.array(list(map(Fraction, arr1)))
frac_arr2 = np.array(list(map(Fraction, arr2)))

frac_arr1/frac_arr2

array([Fraction(1, 2), Fraction(2, 3)], dtype=object)

### Testing All-Slack Starting Method with Fractions

In [12]:
# question 4
# no. of decision variables (N)
NO_OF_DECISION_VARS = 3

# no. of conditions 
NO_OF_CONDITIONS = 3

# --------------------------------- objective function ---------------------------------
# specify the objective function in the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>]
OBJ_FUNC = [Fraction(235, 100), 3, Fraction(285, 100)]

# -------------------------------- type of optimization --------------------------------
OPT_TYPE = "MAX" # or "MIN" 

# ------------------------------------- conditions -------------------------------------
# specify each condition in a new row with the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>, <solution>]
conditions = [
    [ 10, 9, 14, 4_000],
    [  5, 6, 12, 3_000],
    [  0, 1,  0,   250],
]

In [13]:
# convert them to fractions
OBJ_FUNC = np.array(list(map(Fraction, OBJ_FUNC)))

for i in range(len(conditions)):
    conditions[i] = list(map(Fraction, conditions[i]))

conditions = np.array(conditions)

In [14]:
var_symbol_arr = []
basic_var_symbol_arr = []

OBJ_FUNC_VAR_SYMBOL = "P"
DECISION_VAR_SYMBOL = "X"
SLACK_VAR_SYMBOL    = "S"

# ========================= ALL THE VARIABLES =========================
var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_DECISION_VARS):
    var_symbol_arr.append(DECISION_VAR_SYMBOL + str(i+1))

for i in range(NO_OF_CONDITIONS):
    var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("variables:", var_symbol_arr)

# ========================== BASIC VARIABLES ==========================
basic_var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_CONDITIONS):
    basic_var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("basic variables:", basic_var_symbol_arr)

variables: ['P', 'X1', 'X2', 'X3', 'S1', 'S2', 'S3']
basic variables: ['P', 'S1', 'S2', 'S3']


In [16]:
# find the size of a row
row_size = 1 + NO_OF_DECISION_VARS + NO_OF_CONDITIONS + 1

# find the column size
col_size = 1 + NO_OF_CONDITIONS # no. of basic vars = 1 (for objective function) + no. of conditions

# construct the initial tabulation
obj_func_row = np.concatenate(([1], -OBJ_FUNC, np.zeros([NO_OF_CONDITIONS]), [0]))
initial_tabulation = obj_func_row.reshape((1, -1))

for i in range(1, col_size):

    # condition rows
    temp = np.zeros([NO_OF_CONDITIONS])
    temp[i-1] = 1
    row = np.concatenate(([0], conditions[i-1][ :-1], temp, conditions[i-1][-1: ])).reshape((1, -1))

    initial_tabulation = np.concatenate((initial_tabulation, row), axis=0)

In [17]:
# convert all the existing elements to fractions
for i in range(initial_tabulation.shape[0]):
    initial_tabulation[i] = np.array(list(map(Fraction, initial_tabulation[i])))

In [19]:
visualize_tabulation(initial_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

┌────┬─────┬───────┬──────┬───────┬──────┬──────┬──────┬───────┐
│    │   P │    X1 │   X2 │    X3 │   S1 │   S2 │   S3 │   sol │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ P  │   1 │ -2.35 │   -3 │ -2.85 │    0 │    0 │    0 │     0 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │ 10    │    9 │ 14    │    1 │    0 │    0 │  4000 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │  5    │    6 │ 12    │    0 │    1 │    0 │  3000 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S3 │   0 │  0    │    1 │  0    │    0 │    0 │    1 │   250 │
└────┴─────┴───────┴──────┴───────┴──────┴──────┴──────┴───────┘


In [20]:
_ = all_slack_starting(initial_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr, opt_type=OPT_TYPE)

==================== iteration no.: 1 ====================
entering var: X2
pivot col: [Fraction(-3, 1) Fraction(9, 1) Fraction(6, 1) Fraction(1, 1)]
solution : [Fraction(0, 1) Fraction(4000, 1) Fraction(3000, 1) Fraction(250, 1)]
ratio col: [ -0.         444.44444444 500.         250.        ]
leaving var : S3
pivot row: [Fraction(0, 1) Fraction(0, 1) Fraction(1, 1) Fraction(0, 1)
 Fraction(0, 1) Fraction(0, 1) Fraction(1, 1) Fraction(250, 1)]
new basic variables: ['P', 'S1', 'S2', 'X2']
┌────┬─────┬───────┬──────┬───────┬──────┬──────┬──────┬───────┐
│    │   P │    X1 │   X2 │    X3 │   S1 │   S2 │   S3 │   sol │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ P  │   1 │ -2.35 │    0 │ -2.85 │    0 │    0 │    3 │   750 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │ 10    │    0 │ 14    │    1 │    0 │   -9 │  1750 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │  5    │    0 │ 12    │    0 │    1 │  

/home/akila/semester_8/OR/helper.py:69: RuntimeWarning: divide by zero encountered in divide
  ratio_col = np.float64(solution) / np.float64(pivot_col)


In [18]:
Fraction(0)

Fraction(0, 1)

In [10]:
import numpy as np
from fractions import Fraction

from helper import integer_program, visualize_tabulation

tabulation = np.array([
    [1, 0, 9, 0, 11, 0.5, 294],
    [0, 1, 6.25, 0, 4, -1, 36.5],
    [0, 0, -1.75, 1, 1.25, 0.5, 6.75],
])
var_symbol_arr = ["P", "X1", "X2", "X3", "S1", "S2"]
basic_var_symbol_arr = ["P", "X1", "X3"]

tabulation_ = []
for i in range(tabulation.shape[0]):
    tabulation_.append(list(map(Fraction, tabulation[i])))

tabulation_ = np.array(tabulation_)
visualize_tabulation(tabulation_, var_symbol_arr, basic_var_symbol_arr)

┌────┬─────┬──────┬───────┬──────┬───────┬──────┬────────┐
│    │   P │   X1 │    X2 │   X3 │    S1 │   S2 │    sol │
├────┼─────┼──────┼───────┼──────┼───────┼──────┼────────┤
│ P  │   1 │    0 │  9    │    0 │ 11    │  0.5 │ 294    │
├────┼─────┼──────┼───────┼──────┼───────┼──────┼────────┤
│ X1 │   0 │    1 │  6.25 │    0 │  4    │ -1   │  36.5  │
├────┼─────┼──────┼───────┼──────┼───────┼──────┼────────┤
│ X3 │   0 │    0 │ -1.75 │    1 │  1.25 │  0.5 │   6.75 │
└────┴─────┴──────┴───────┴──────┴───────┴──────┴────────┘


In [11]:
integer_program(tabulation_, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr, opt_type="MAX")

############################## ITERATION NO. 1 ##############################
max. frac: 3/4
resource var: X3
┌────┬─────┬──────┬───────┬──────┬───────┬──────┬──────┬────────┐
│    │   P │   X1 │    X2 │   X3 │    S1 │   S2 │   G1 │    sol │
├────┼─────┼──────┼───────┼──────┼───────┼──────┼──────┼────────┤
│ P  │   1 │    0 │  9    │    0 │ 11    │  0.5 │    0 │ 294    │
├────┼─────┼──────┼───────┼──────┼───────┼──────┼──────┼────────┤
│ X1 │   0 │    1 │  6.25 │    0 │  4    │ -1   │    0 │  36.5  │
├────┼─────┼──────┼───────┼──────┼───────┼──────┼──────┼────────┤
│ X3 │   0 │    0 │ -1.75 │    1 │  1.25 │  0.5 │    0 │   6.75 │
├────┼─────┼──────┼───────┼──────┼───────┼──────┼──────┼────────┤
│ G1 │   0 │    0 │  0.75 │    0 │ -0.25 │ -0.5 │    1 │  -0.75 │
└────┴─────┴──────┴───────┴──────┴───────┴──────┴──────┴────────┘

APPLYING DUAL SIMPLEX...
====================DUAL SIMPLEX - iteration no.: 1====================
leaving variale : G1
ratio row       : [ inf  nan  12.  nan  44.  

/home/akila/semester_8/OR/helper.py:152: RuntimeWarning: divide by zero encountered in divide
  ratio_row = np.float64(np.abs(tabulation[0])) / np.float64(np.abs(tabulation[pivot_row_idx]))
/home/akila/semester_8/OR/helper.py:152: RuntimeWarning: invalid value encountered in divide
  ratio_row = np.float64(np.abs(tabulation[0])) / np.float64(np.abs(tabulation[pivot_row_idx]))


In [13]:
tabulation = np.array([
    [1, 4, 1, 0, 0.5, 1, -2, 0, 12],
    [0, 1.5, 5.5, 0, 1, -0.5, 0.5, 0, 25],
    [0, -0.5, 0.5, 1, 0, -0.5, 0.5, 0, 7],
    [0, 4, 1, 0, 0, 1, -1, 1, 12], 
])
var_symbol_arr = ["P", "X1", "X2", "X3", "S1", "S2", "A1", "A2"]
basic_var_symbol_arr = ["Z", "S1", "X3", "A2"]

tabulation_ = []
for i in range(tabulation.shape[0]):
    tabulation_.append(list(map(Fraction, tabulation[i])))

tabulation_ = np.array(tabulation_)
visualize_tabulation(tabulation_, var_symbol_arr, basic_var_symbol_arr)

┌────┬─────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬───────┐
│    │   P │   X1 │   X2 │   X3 │   S1 │   S2 │   A1 │   A2 │   sol │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ Z  │   1 │  4   │  1   │    0 │  0.5 │  1   │ -2   │    0 │    12 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │  1.5 │  5.5 │    0 │  1   │ -0.5 │  0.5 │    0 │    25 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ X3 │   0 │ -0.5 │  0.5 │    1 │  0   │ -0.5 │  0.5 │    0 │     7 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ A2 │   0 │  4   │  1   │    0 │  0   │  1   │ -1   │    1 │    12 │
└────┴─────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴───────┘


In [17]:
from helper import all_slack_starting

_ = all_slack_starting(tabulation_, var_symbol_arr, basic_var_symbol_arr, opt_type="MIN")

==================== iteration no.: 1 ====================
entering var: X1
pivot col: [Fraction(4, 1) Fraction(3, 2) Fraction(-1, 2) Fraction(4, 1)]
solution : [Fraction(12, 1) Fraction(25, 1) Fraction(7, 1) Fraction(12, 1)]
ratio col: [  3.          16.66666667 -14.           3.        ]
leaving var : S2
pivot row: [Fraction(0, 1) Fraction(4, 1) Fraction(1, 1) Fraction(0, 1)
 Fraction(0, 1) Fraction(1, 1) Fraction(-1, 1) Fraction(1, 1)
 Fraction(12, 1)]
new basic variables: ['Z', 'S1', 'X3', 'X1']
┌────┬─────┬──────┬───────┬──────┬──────┬────────┬────────┬────────┬───────┐
│    │   P │   X1 │    X2 │   X3 │   S1 │     S2 │     A1 │     A2 │   sol │
├────┼─────┼──────┼───────┼──────┼──────┼────────┼────────┼────────┼───────┤
│ Z  │   1 │    0 │ 0     │    0 │  0.5 │  0     │ -1     │ -1     │   0   │
├────┼─────┼──────┼───────┼──────┼──────┼────────┼────────┼────────┼───────┤
│ S1 │   0 │    0 │ 5.125 │    0 │  1   │ -0.875 │  0.875 │ -0.375 │  20.5 │
├────┼─────┼──────┼───────┼──────